<a href="https://colab.research.google.com/github/KhondamirRustamov/FoldCraft/blob/main/FoldCraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FoldCraft** - fold-conditioned de novo binder design

<img src="https://github.com/KhondamirRustamov/FoldCraft/blob/main/png/Fig1-1.png?raw=true">

Fold-conditioned binder design using **AF2-Multimer** backpropagation and **ProteinMPNN**.

Select the template structure for binder fold-conditioning. **FoldCraft** creates a target *cmap* using provided template, which will be used for binder design to specified target protein.

if you to design de novo single domain nanobodies using **VHH** framework just use this [notebook](https://colab.research.google.com/github/KhondamirRustamov/FoldCraft/blob/main/FoldCraft_VHH.ipynb).



In [ ]:
#@title setup
%%time
import os

if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")
os.system('pip -q install mdanalysis')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

In [ ]:
#@title Prepare fold conditioned binder | Construct target cmap for binder-target complex
from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1
#@markdown Prepare fold conditioned binder
binder_template = '1QYS' #@param {type:"string"}
chain_template = 'A' #@param {type:"string"}
hotspots = ''
template_pdb = get_pdb(binder_template)

pdbparser = PDBParser()

structure = pdbparser.get_structure(binder_template, template_pdb)
chains = {chain.id:seq1(''.join(residue.resname for residue in chain)) for chain in structure.get_chains()}

query_chain = chains[chain_template]

af_model = mk_afdesign_model(protocol="fixbb", use_templates=True)
af_model.prep_inputs(pdb_filename=template_pdb,
                     ignore_missing=False,
                     chain = chain_template,
                     rm_template_seq=False,
                     rm_template_sc=False,)

#name='9had'
af_model.set_seq(query_chain[:af_model._len])
af_model.predict(num_recycles=3, verbose=False)
print(f"CMAP of {binder_template} (monomer plddt: {af_model.aux['log']['plddt']:.3f})")
#plt.imshow(af_model.aux['cmap'])

import matplotlib.pyplot as plt
import MDAnalysis as mda

import warnings
warnings.filterwarnings("ignore")
#@markdown Prepare target protein structure
pdb_target = 'Q9NZQ7' #@param {type:"string"}
#@markdown Select chain for target protein. If you want to **trim** the protein use **":"** after the chain id and write the **start-end** of trimming region using "-"
chain_id = 'A:19-132' #@param {type:"string"}

pdb_target_path = get_pdb(pdb_target)

def set_range(hotspots_input):
  new_h = [x for x in hotspots_input.split(',')]
  h_range = []
  for i in new_h:
    if '-' in i:
      h_range += [x for x in range(int(i.split('-')[0]), int(i.split('-')[1]))]
    else:
      h_range.append(int(i))
  return h_range

#@markdown Choose hotspots on target protein
target_hotspots = '36-41,84-88,92-96' #@param {type:"string"}
#@markdown Optional: Choose hotspots on binder protein
binder_hotspots = '25-40,50-65' #@param {type:"string"}

if ':' in chain_id:
  chain_cut = [int(x) for x in chain_id.split(':')[-1].split('-')]
  chain_id = chain_id.split(':')[0]
  uref = mda.Universe(f'AF-{pdb_target}-F1-model_v3.pdb')
  uref = uref.select_atoms(f'resid {chain_cut[0]}:{chain_cut[1]}')
  uref.write(f"AF-{pdb_target}-F1-model_v3.pdb")
  target_hotspots_np = np.array(set_range(target_hotspots))-chain_cut[0]
else:
  target_hotspots_np = np.array(set_range(target_hotspots))

af_target = mk_afdesign_model(protocol="fixbb", use_templates=True)
af_target.prep_inputs(pdb_filename=pdb_target_path,
                     ignore_missing=False,
                     chain = chain_id,)

target_len = af_target._len
binder_len = af_model._len

load_np = af_model.aux['cmap']

fc_cmap = np.zeros((target_len+binder_len, target_len+binder_len))

if binder_hotspots == '':
  cdr_range = np.array([range(0,binder_len)])+target_len
else:
  cdr_range = np.array(set_range(binder_hotspots))+target_len

fc_cmap[-binder_len:,-binder_len:] = load_np


for i in target_hotspots_np:
    for x in cdr_range:
        fc_cmap[x,i] = 1.
        fc_cmap[i,x] = 1.

from matplotlib import patches
fig, ax = plt.subplots()
plt.imshow(fc_cmap,cmap='grey_r')
rect = patches.Rectangle((0, 0), target_len, target_len, linewidth=2, edgecolor='b', facecolor='none')
rect2 = patches.Rectangle((target_len, target_len), binder_len, binder_len, linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(rect)
ax.add_patch(rect2)
print('Target (blue)')
print('Binder (red)')

folder_name = f'{binder_template}_{pdb_target}'
os.system(f'mkdir {folder_name}')
np.save(f'{folder_name}/fold_cond_cmap.npy',fc_cmap)

fc_cmap[fc_cmap>0] = 1
np.save(f'{folder_name}/fold_cond_cmap_mask.npy',fc_cmap)

In [ ]:
#@title Define fold-conditioned loss and run 3stage design (100,100,20)
from colabdesign.af.loss import get_contact_map
import jax
import jax.numpy as jnp

conditioned_array = np.load('fold_cond_cmap.npy')
conditioned_mask = np.load('fold_cond_cmap_mask.npy')

def cmap_loss_binder(inputs, outputs):
    global conditioned_array
    global conditioned_mask
    global binder_len

    i_cmap = get_contact_map(outputs, inputs["opt"]["i_con"]["cutoff"])
    cmap = get_contact_map(outputs, inputs["opt"]["con"]["cutoff"])
    i_cmap = i_cmap.at[-binder_len:,-binder_len:].set(cmap[-binder_len:,-binder_len:])
    out_cmap_conditioned = i_cmap * conditioned_mask
    cmap_loss_binder = jnp.sqrt(jnp.square(out_cmap_conditioned - conditioned_array).sum(-1).mean())

    return {"cmap_loss_binder":cmap_loss_binder}

name = 'tmp_0' #@param {type:"string"}
rm_aa = 'C' #@param {type:"string"}

af_model_design = mk_afdesign_model(protocol="binder", loss_callback=cmap_loss_binder,
                                     use_templates=True,)

af_model_design.prep_inputs(pdb_filename=pdb_target_path,
                             chain=chain_id, binder_len = binder_len,
                             hotspot=target_hotspots,
                             rm_aa=rm_aa, #fix_pos=fixed_positions,
                             )

af_model_design.opt["weights"]["cmap_loss_binder"] = 1.
af_model_design.opt["weights"].update({"cmap_loss_binder":1.0, "rmsd":0.0, "fape":0.0, "plddt":0.0,
                                        "con":0.0, "i_con":0.0, "i_pae":0.})

af_model_design.design_3stage(100,100,20)

In [ ]:
#@title Optimize sequences with ProteinMPNN
import pickle
from colabdesign.mpnn import mk_mpnn_model
import pandas as pd
from Bio.PDB import PDBParser, DSSP, Selection, Polypeptide, PDBIO, Select, Chain, Superimposer
from scipy.spatial import cKDTree

n_samples = 5  #@param {type:"integer"}
remove_aa = 'C'  #@param {type:"string"}
model_name = "v_48_010"
weights = "soluble" #@param ["original", 'soluble'] {type:"raw"}
redesign_strategy = "non-interface" #@param ["full", 'non-interface'] {type:"raw"}

three_to_one_map = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

def hotspot_residues(trajectory_pdb, binder_chain="B", atom_distance_cutoff=4.0):
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", trajectory_pdb)

    # Get the specified chain
    binder_atoms = Selection.unfold_entities(structure[0][binder_chain], 'A')
    binder_coords = np.array([atom.coord for atom in binder_atoms])

    # Get atoms and coords for the target chain
    target_atoms = Selection.unfold_entities(structure[0]['A'], 'A')
    target_coords = np.array([atom.coord for atom in target_atoms])

    # Build KD trees for both chains
    binder_tree = cKDTree(binder_coords)
    target_tree = cKDTree(target_coords)

    # Prepare to collect interacting residues
    interacting_residues = {}

    # Query the tree for pairs of atoms within the distance cutoff
    pairs = binder_tree.query_ball_tree(target_tree, atom_distance_cutoff)

    # Process each binder atom's interactions
    for binder_idx, close_indices in enumerate(pairs):
        binder_residue = binder_atoms[binder_idx].get_parent()
        binder_resname = binder_residue.get_resname()

        # Convert three-letter code to single-letter code using the manual dictionary
        if binder_resname in three_to_one_map:
            aa_single_letter = three_to_one_map[binder_resname]
            for close_idx in close_indices:
                target_residue = target_atoms[close_idx].get_parent()
                interacting_residues[binder_residue.id[1]] = aa_single_letter

    return interacting_residues
os.system(f'mkdir {folder_name}/mpnn')

af_model_design.save_pdb(f"{folder_name}/{name}.pdb", get_best=False)

with open(f'{folder_name}/{name}.pickle', 'wb') as handle:
  pickle.dump(af_model_design.aux['all'], handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Running ProteinMPNN...')

design_pos = [f'{i}' for i in range(1,binder_len)]

if redesign_strategy == "non-interface":
  interface_residues_list = list(hotspot_residues(f"{name}.pdb", 'B').keys())
  new_design_pos = ','.join([f'B{i}' for i in design_pos if i not in interface_residues_list])
else:
  new_design_pos = ','.join([f'B{i}' for i in design_pos])

mpnn_model = mk_mpnn_model(model_name, backbone_noise=0.0,weights=weights)
mpnn_model.prep_inputs(pdb_filename=f"{folder_name}/{name}.pdb", chain='A,B', fix_pos=new_design_pos, rm_aa = remove_aa, inverse=True)

samples = mpnn_model.sample_parallel(temperature=0.1, batch=n_samples)

print('Predicting sequences with AF2_ptm...')

names = []
sequences = []
plddts = []
ipaes = []
iptms = []
cmap_loss = []

for num, seq in enumerate(samples['seq']):
  af_model_predict = mk_afdesign_model(protocol="binder", loss_callback=cmap_loss_binder,
                                       use_templates=True,)

  af_model_predict.prep_inputs(pdb_filename=pdb_target_path,
                               chain=chain_id, binder_len = binder_len,
                               hotspot=target_hotspots,
                               rm_aa=rm_aa, #fix_pos=fixed_positions,
                               )
  af_model_predict.set_seq(seq[-binder_len:])
  af_model_predict.predict(num_recycles=3, verbose=False, models=["model_1_ptm","model_2_ptm"])
  print(f"predict: {name}_{num} plddt: {af_model_predict.aux['log']['plddt']:.3f}, i_pae: {(af_model_predict.aux['log']['i_pae']):.3f}, i_ptm: {af_model_predict.aux['log']['i_ptm']:.3f}, cmap_loss: {af_model_predict.aux['log']['cmap_loss_binder']:.3f}")
  af_model_predict.save_pdb(f"{folder_name}/mpnn/{name}_{num}.pdb", get_best=False)
  with open(f'{folder_name}/mpnn/{name}_{num}.pickle', 'wb') as handle:
    pickle.dump(af_model_predict.aux['all'], handle, protocol=pickle.HIGHEST_PROTOCOL)
  names.append(f'{name}_{num}')
  sequences.append(seq)
  plddts.append(af_model_predict.aux['log']['plddt'])
  ipaes.append(af_model_predict.aux['log']['i_pae'])
  iptms.append(af_model_predict.aux['log']['i_ptm'])
  cmap_loss.append(af_model_predict.aux['log']['cmap_loss_binder'])

df = pd.DataFrame({'name':names,
                   'sequence':sequences,
                   'plddt':plddts,
                   'ipae':ipaes,
                   'iptm':iptms,
                   'cmap_loss':cmap_loss})
df.to_csv(f"{folder_name}/{name}.csv")

In [ ]:
#@title Plot the best design (by cmap loss)
import py3Dmol

df = df.sort_values(by='cmap_loss')

with open(f"{folder_name}/{df['name'].tolist()[0]}.pdb", "r") as file:
    pdb_data = file.read()
color = "lDDT"

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(pdb_data, "pdb")

if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
elif color == "chain":
    chains = len(queries[0][1]) + 1 if is_complex else 1
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})

view.zoomTo()
view.show()

In [ ]:
#@title Animate the hallucination process
HTML(af_model_design.animate())

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive,
#@markdown try disabling your adblocker and run this cell again.
#@markdown  If that fails click on the little folder icon to the
#@markdown  left, navigate to file: `{folder_name}.result.zip`,
#@markdown  right-click and select \"Download\"
!zip -r {folder_name}.result.zip {folder_name}/*
files.download(f"{folder_name}.result.zip")